# Install the Package
Here we're installing it directly from GitHub while it's in development.

In [1]:
!pip install 'vanna[flask,anthropic]'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.2/388.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.5/485.5 kB 23.2 MB/s eta 0:00:00


# Download a Sample Database

In [2]:
import httpx

with open("Chinook.sqlite", "wb") as f:
    with httpx.stream("GET", "https://vanna.ai/Chinook.sqlite") as response:
        for chunk in response.iter_bytes():
            f.write(chunk)

# Imports

In [8]:
from vanna import Agent, AgentConfig
from vanna.servers.fastapi import VannaFastAPIServer
from vanna.core.registry import ToolRegistry
from vanna.core.user import UserResolver, User, RequestContext
from vanna.integrations.openai import OpenAILlmService
from vanna.tools import RunSqlTool, VisualizeDataTool
from vanna.integrations.sqlite import SqliteRunner
from vanna.tools.agent_memory import SaveQuestionToolArgsTool, SearchSavedCorrectToolUsesTool
from vanna.integrations.local.agent_memory import DemoAgentMemory
from vanna.capabilities.sql_runner import RunSqlToolArgs
from vanna.tools.visualize_data import VisualizeDataArgs

# Define your User Authentication
Here we're going to say that if you're logged in as `admin@example.com` then you're in the `admin` group, otherwise you're in the `user` group

In [9]:
class SimpleUserResolver(UserResolver):
    async def resolve_user(self, request_context: RequestContext) -> User:
        # In production, validate cookies/JWTs here
        user_email = request_context.get_cookie('vanna_email')
        if not user_email:
            raise ValueError("Missing 'vanna_email' cookie for user identification")

        print(f"Resolving user for email: {user_email}")

        if user_email == "admin@example.com":
            return User(id="admin1", email=user_email, group_memberships=['admin'])

        return User(id="user1", email=user_email, group_memberships=['user'])

# Define the Tools and Access Control

In [10]:
tools = ToolRegistry()
tools.register_local_tool(RunSqlTool(sql_runner=SqliteRunner(database_path="./Chinook.sqlite")), access_groups=['admin', 'user'])
tools.register_local_tool(VisualizeDataTool(), access_groups=['admin', 'user'])
agent_memory = DemoAgentMemory(max_items=1000)
tools.register_local_tool(SaveQuestionToolArgsTool(), access_groups=['admin'])
tools.register_local_tool(SearchSavedCorrectToolUsesTool(), access_groups=['admin', 'user'])

In [ ]:
# Set up LLM
llm = OpenAILlmService(
    model="Qwen/Qwen3-8B",
    api_key="sk-upbzqrghdrybqxmceluybpbwixtjrnqkqdvbfwmsxkocmerr",
    base_url="https://api.siliconflow.cn/v1"
)

# Create agent with your options
agent = Agent(
    llm_service=llm,
    tool_registry=tools,
    user_resolver=SimpleUserResolver(),
    config=AgentConfig(),
    agent_memory=agent_memory
)

# 4. Create and run server
server = VannaFastAPIServer(agent)
server.run()

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-1' coro=<Server.serve() done, defined at /usr/local/lib/python3.12/dist-packages/uvicorn/server.py:69> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-106055717.py", line 15, in <cell line: 0>
    server.run()
  File "/usr/local/lib/python3.12/dist-packages/vanna/servers/fastapi/app.py", line 160, in run
    asyncio.run(server.serve())
  File "/usr/local/lib/python3.12/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.12/dist-packages/nest_asyncio.py", line 133, in _run_once
   

Try `serve_kernel_port_as_iframe` instead. 


<IPython.core.display.Javascript object>

Your app is running at:
https://8000-m-s-1a63ot0yjfe32-b.us-east1-0.prod.colab.dev


INFO:     Started server process [638]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:36022 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:36028 - "POST /api/vanna/v2/chat_sse HTTP/1.1" 200 OK


ERROR:vanna.core.agent.agent:Error in send_message (conversation_id=1766659993753-u3uo2ep8k): Missing 'vanna_email' cookie for user identification
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/vanna/core/agent/agent.py", line 162, in send_message
    async for component in self._send_message(
  File "/usr/local/lib/python3.12/dist-packages/vanna/core/agent/agent.py", line 257, in _send_message
    user = await self.user_resolver.resolve_user(request_context)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-1146226105.py", line 6, in resolve_user
    raise ValueError("Missing 'vanna_email' cookie for user identification")
ValueError: Missing 'vanna_email' cookie for user identification
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/vanna/core/agent/agent.py", line 162, in send_message
    async for component in self._send_message(
  File "/usr/local/lib/python3.12/dist-pa

INFO:     127.0.0.1:36030 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:39164 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:39170 - "POST /api/vanna/v2/chat_sse HTTP/1.1" 200 OK
Resolving user for email: admin@example.com
INFO:     127.0.0.1:51038 - "POST /api/vanna/v2/chat_sse HTTP/1.1" 200 OK
Resolving user for email: admin@example.com
INFO:     127.0.0.1:36732 - "POST /api/vanna/v2/chat_sse HTTP/1.1" 200 OK
Resolving user for email: admin@example.com
